In [1]:
import mne
import skada
import sys
import os
sys.path.append('C:/Users/s.velut/Documents/These/Protheus_PHD/Scripts')
sys.path.append('C:/Users/s.velut/Documents/These/Protheus_PHD/Scripts/Wavelets/Green_files/research_code/')
from Wavelets.Green_files.green.data_utils import EpochsDataset
from Wavelets.Green_files.green.wavelet_layers import RealCovariance, PW_PLV
import torch
from tensorflow import keras
from scikeras.wrappers import KerasClassifier
from scipy import stats


from EEG2CodeKeras import (basearchi,
                           basearchitest_batchnorm,
                           basearchi_patchembedding,
                           basearchi_patchembeddingdilation,
                           trueVanilliaEEG2Code,
                           vanilliaEEG2Code,
                           vanilliaEEG2Code2,
                           EEGnet_Inception)

from Wavelets.Green_files.research_code.pl_utils import get_green, GreenClassifierLM
from Wavelets.Green_files.research_code.crossval_utils import pl_crossval
from Wavelets.Green_files.tests.conftest import make_one_dummy_epoch
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


mne.set_log_level('ERROR')



c:\Users\s.velut\AppData\Local\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
from utils import get_BVEP_data, prepare_data, balance
from _utils import make_preds_accumul_aggresive
sys.path.append('C:/Users/s.velut/Documents/These/Protheus_PHD/Scripts/Wavelets/Jade/')
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\datasets")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\paradigms")
import numpy as np
from copy import deepcopy
import time
from sklearn.pipeline import make_pipeline
from pyriemann.utils.mean import mean_covariance
from pyriemann.tangentspace import TangentSpace
from pyriemann.estimation import Xdawn, XdawnCovariances
from sklearn.metrics import balanced_accuracy_score,f1_score,recall_score,confusion_matrix
from sklearn.model_selection import train_test_split
from SPDNet.torch.optimizers import riemannian_adam as torch_riemannian_adam
from Alignments.covariance import compute_covariances
from Alignments.riemannian import compute_riemannian_alignment,compute_ref_riemann
from torch.utils.data import DataLoader, TensorDataset

from castillos2023 import CasitllosBurstVEP100

c:\Users\s.velut\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


# Test sur la librairie skada

In [3]:
participants = ['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10',
                'P11','P12','P13','P14','P15','P16','P17','P18','P19','P20',
                'P21','P22','P23','P24']
path = 'C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Data\\Dry_Ricker'
# path = '/home/dcas/k.cabrera/Data/SETNOP2'
nb_subject = len(participants)
n_class=5
fmin = 1
fmax = 45
fps = 60
window_size = 0.35

sfreq = 500
raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
               for i in range(len(participants))]

C:\Users\s.velut\AppData\Local\Temp\ipykernel_9180\1570998499.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9180\1570998499.py:14: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9180\1570998499.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Use

In [4]:
for ind_i,i in enumerate(participants):
    # raw = raw.drop_channels([ch for ch in raw.ch_names if ch in to_drop])
    # raw = raw.drop_channels([i for i in raw.ch_names if i not in keep])

    raw_eeglab[ind_i] = raw_eeglab[ind_i].filter(l_freq=fmin, h_freq=fmax, method="fir", verbose=True)

    # mne.set_eeg_reference(raw_eeglab[ind_i], 'average', copy=False, verbose=False)
    n_channels = len(raw_eeglab[ind_i].ch_names)
    print("Channels :", raw_eeglab[ind_i].ch_names)
    # print(raw_eeglab[ind_i].info)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Wi

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

In [5]:
epochs_list = []
events_list = []
events_id_list = []
onset_code_list = []
data_list = []
labels_code_list = []
for ind_i, i in enumerate(participants): 
    # Strip the annotations that were script to make them easier to process
    events, event_id = mne.events_from_annotations(raw_eeglab[ind_i], event_id='auto', verbose=False)
    to_remove = []
    for idx in range(len(raw_eeglab[ind_i].annotations.description)):
        if (('boundary' in raw_eeglab[ind_i].annotations.description[idx]) or
            ('BURST' in raw_eeglab[ind_i].annotations.description[idx])):
            to_remove.append(idx)

    to_remove = np.array(to_remove)
    if len(to_remove) > 0:
        raw_eeglab[ind_i].annotations.delete(to_remove)
    # Get the events
    temp_event,temp_event_id = mne.events_from_annotations(raw_eeglab[ind_i], event_id='auto', verbose=False)
    events_list.append(temp_event)
    events_id_list.append(temp_event_id)
    shift = 0.0
    # Epoch the data following event
    epochs_list.append(mne.Epochs(raw_eeglab[ind_i], events_list[ind_i], event_id=events_id_list[ind_i], tmin=shift, \
                tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False))
    # print(events_list[ind_i])

    labels_code_list.append(epochs_list[ind_i].events[..., -1])
    labels_code_list[ind_i] -= np.min(labels_code_list[ind_i])
    # print(epochs.events[..., -1])
    data_list.append(epochs_list[ind_i].get_data())
    info_ep = epochs_list[ind_i].info
    # print(epochs)
    onset_code_list.append(epochs_list[ind_i].events[..., 0])
data_list = np.array(data_list)

In [6]:
from collections import OrderedDict
codes = OrderedDict()
for k, v in events_id_list[0].items():
    code = k.split('_')[0]
    code = code.replace('.','').replace('2','')
    idx = k.split('_')[1]
    codes[v-1] = np.array(list(map(int, code)))

In [7]:
def to_window_old(data, labels,length,n_samples_windows,codes,window_size=0.25,normalise=True,sfreq=500,fps=60,n_channels=8):
    
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    idx_taken = []
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1
        
        for idx in range(length):
            # print('Xidx:', trial_nb*length+idx, "Tidxm:", idx, 'TidxM:', idx +
            #       n_samples_windows, 'Ltrial', trial[:, idx:idx+n_samples_windows].shape)
            idx_taken.append(trial_nb*length+idx)
    # if normalise:
    #     X_std = X.std(axis=0)
    #     X /= X_std + 1e-8
    y_pred = np.vstack((y,np.abs(1-y))).T
    y = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])

    return X, y, np.array(idx_taken)

In [8]:

n_samples_windows = int(window_size*sfreq)
# length = int((2.2-window_size)*sfreq)
length = int((2.2-window_size)*fps)
X = np.zeros((data_list.shape[0],length*data_list.shape[1],data_list.shape[2],n_samples_windows))
Y = np.zeros((data_list.shape[0],length*data_list.shape[1]))
idx_taken = np.zeros((data_list.shape[0],length*data_list.shape[1]))
domains = []
print(Y.shape)
print(X.shape)
for ind_i,i in enumerate(participants):
    # X[ind_i],y[ind_i],idx_taken[ind_i] = to_window_old(data_list[ind_i],labels_code_list[ind_i],length,n_samples_windows,codes,window_size=window_size,sfreq=sfreq)
    temp_X,Y[ind_i],idx_taken[ind_i] = to_window_old(data_list[ind_i],labels_code_list[ind_i],length,n_samples_windows,codes,window_size=window_size,sfreq=fps)
    domains.append(["Source_sub_{}".format(ind_i+1),]*len(Y[ind_i]))
    # X[ind_i] = compute_riemannian_alignment(temp_X, mean=None, dtype='real')

# print(codes[labels_test[0]])
print(Y.shape)
print(X.shape)
domains = np.array(domains)

(24, 8325)
(24, 8325, 8, 175)
(24, 8325)
(24, 8325, 8, 175)


In [9]:
def onset_anno(onset_window,label_window,onset_code,nb_seq_min,nb_seq_max,code_freq,sfreq,win_size):
    assert(sfreq!=0)
    new_onset = []
    new_onset_0 = []
    current_code = 0
    onset_code = np.ceil(onset_code*code_freq/sfreq)
    nb_seq_min-=1
    onset_shift = onset_code[current_code+nb_seq_min]
    time_trial = (2.2-win_size)
    # onset_window = np.arange(0,time_trial*code_freq*(nb_seq_max-nb_seq_min)-1,1,dtype=int)
    for i,o in enumerate(onset_window):
        if label_window[i]==1:
            # print(i)
            if current_code==nb_seq_max-1-nb_seq_min:
                new_onset.append(o+onset_shift)
            else:
                if o+onset_shift >= onset_code[current_code+nb_seq_min]+time_trial*code_freq:
                    current_code+=1
                    onset_shift = onset_code[current_code+nb_seq_min]-time_trial*code_freq*current_code
                new_onset.append(o+onset_shift)
        else:
            if current_code==nb_seq_max-1-nb_seq_min:
                new_onset_0.append(o+onset_shift)
            else:
                if o+onset_shift >= onset_code[current_code+nb_seq_min]+time_trial*code_freq:
                    current_code+=1
                    onset_shift = onset_code[current_code+nb_seq_min]-time_trial*code_freq*current_code
                new_onset_0.append(o+onset_shift)
    
    # modified_onset_code = [onset_code[i]-time_trial*sfreq*i for i in range(nb_seq_min,nb_seq_max)]
    # new_onset_0 = np.concatenate([np.arange(onset_code[i],onset_code[i]+time_trial*sfreq,sfreq//60) for i in range(nb_seq_min,nb_seq_max)])
    new_onset_0 = np.array(list(filter(lambda i: i not in new_onset, new_onset_0)))
    # print(new_onset_0.shape)
    return np.array(new_onset)/code_freq, np.array(new_onset_0)/code_freq
            

In [10]:

for ind_i,i in enumerate(participants):
    onset,onset_0 = onset_anno(idx_taken[ind_i],Y[ind_i],onset_code_list[ind_i],1,n_class*15,60,500,window_size)
    # print(onset,onset_0)
    anno = mne.Annotations(onset,0.001*sfreq//60,"1")
    anno.append(onset_0,0.001*sfreq//60,"0")

    raw_eeglab[ind_i] = raw_eeglab[ind_i].set_annotations(anno)

X_parent = np.zeros((X.shape[0],length*data_list.shape[1],data_list.shape[2],int(window_size*sfreq)))
domains_parent = []
Y_parent = np.zeros((data_list.shape[0],length*data_list.shape[1]))
for ind_i,i in enumerate(participants):
    events, event_id = mne.events_from_annotations(raw_eeglab[ind_i])
    # print(event_id)
    epochs = mne.Epochs(raw_eeglab[ind_i],events,event_id,tmin=0.0,tmax=window_size,baseline=(0,0))
    X_parent[ind_i] = epochs.get_data()[:,:,:-1]
    Y_parent[ind_i] = epochs.events[...,-1]-1

    # xdawn = Xdawn(nfilter=8,classes=[1],estimator='lwf')
    # temp_X = xdawn.fit_transform(temp_X,Y_parent[ind_i])
    # temp_X = np.hstack([temp_X,np.tile(xdawn.evokeds_[None,:,:],(temp_X.shape[0],1,1))])
    
    # X_parent[ind_i] = compute_riemannian_alignment(temp_X, mean=None, dtype='real')

    
    
    domains_parent.append([ind_i+1]*len(Y_parent[ind_i]))
    # domains_parent.append(["Source_sub_{}".format(ind_i+1),]*len(Y_parent[ind_i]))

domains_parent = np.array(domains_parent)

In [11]:
model = get_green(
    n_freqs=20,
    kernel_width_s=window_size,
    n_ch=16,
    sfreq=500,
    oct_min=0,
    oct_max=4.5,
    orth_weights=False,
    dropout=.7,
    hidden_dim=[20,10],
    logref='logeuclid',
    pool_layer=RealCovariance(),
    bi_out=[8,4],
    dtype=torch.float32,
    out_dim=2,
    use_age=False,
)
model_pl = GreenClassifierLM(model=model,
                            criterion=torch.nn.CrossEntropyLoss(),)
model_pl

GreenClassifierLM(
  (model): Green(
    (conv_layers): Sequential(
      (0): ComplexWavelet(kernel_width_s=0.35, sfreq=500, n_wavelets=20, stride=5, padding=0, scaling=oct)
    )
    (pooling_layers): RealCovariance()
    (spd_layers): Sequential(
      (0): LedoitWold(n_freqs=20, init_shrinkage=-3.0, learnable=True)
      (1): BiMap(d_in=16, d_out=8, n_freqs=20
      (2): BiMap(d_in=8, d_out=4, n_freqs=20
    )
    (proj): LogEig(ref=logeuclid, reg=0.0001, n_freqs=20, size=4
    (head): Sequential(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.7, inplace=False)
      (2): Linear(in_features=200, out_features=20, bias=True)
      (3): GELU(approximate='none')
      (4): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Dropout(p=0.7, inplace=False)
      (6): Linear(in_features=20, out_features=10, bias=True)
      (7): GELU(approximate='none')
      (8): BatchNorm1d(10, eps=

In [32]:
def cosine_similarity(X1,X2):
    X1 = np.reshape(X1,(X1.shape[0],X1.shape[1]*X1.shape[2]))
    X2 = np.reshape(X2,(X2.shape[0],X2.shape[1]*X2.shape[2]))
    X1_mean = np.mean(X1)
    X2_mean = np.mean(X2)

    num = np.dot(X1-X1_mean,X2-X2_mean)
    den = np.linalg.norm(X1-X1_mean,axis=1)*np.linalg.norm(X2-X2_mean,axis=1)

    cos_sim = num/den

    return np.median(cos_sim)

def get_interrested_corr(corr_stat):
    length = corr_stat.shape[0]
    return np.array([corr_stat[i,length//2+i] for i in range(length//2)])

def get_most_corr(X,Y,domains,ind_test,M):
    corr_list = []
    print(X.shape)
    assert M < X.shape[0], "M need to be stricly inferior to number of participant"
    for j in range(X.shape[0]):
        X_test = X[ind_test]
        if j!=ind_test:       

            corr = stats.spearmanr(np.mean(X[j],axis=0),np.mean(X_test,axis=0),axis=1)
            print("intrresting corr{}".format(j),get_interrested_corr(corr.statistic))
            corr_list.append(np.mean(get_interrested_corr(corr.statistic)))
        else:
            corr_list.append(-10)
    
    print("list of correlation with sub {}".format(ind_test),corr_list)
    max_ind = np.argpartition(np.abs(corr_list), -M)[-M:]-1

    return X[max_ind],Y[max_ind],domains[max_ind]

# get_most_corr(X_parent,Y_parent,domains_parent,9,6)

In [34]:
max_sub = 6
Y = Y_parent.copy()
X = np.zeros((X_parent.shape[0],X_parent.shape[1],X_parent.shape[2],X_parent.shape[3]))
i = 9
domains = domains_parent.copy()

# #preprocessDA
n_cal = 4
nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*60)
for j in range(nb_subject):
    xdawn = Xdawn(nfilter=4,classes=[1],estimator='lwf')
    if j==i:
        X_std = X_parent[j][:nb_sample_cal].std(axis=0)
        temp_X = X_parent[j]/(X_std + 1e-8)
        xdawn = xdawn.fit(temp_X[:nb_sample_cal],Y[j][:nb_sample_cal])
        temp_X_train = xdawn.transform(temp_X[:nb_sample_cal])
        temp_Xtest = xdawn.transform(temp_X[nb_sample_cal:])
        X[j][:nb_sample_cal] = np.hstack([temp_X_train,np.tile(xdawn.evokeds_[None,:,:],(temp_X_train.shape[0],1,1))])
        X[j][nb_sample_cal:] = np.hstack([temp_Xtest,np.tile(xdawn.evokeds_[None,:,:],(temp_Xtest.shape[0],1,1))])
        # rmean = mean_covariance(compute_covariances(X[j][:nb_sample_cal], estimator='lwf'), metric='riemann')
        # X[j] = compute_riemannian_alignment(X[j], mean=rmean, dtype='real')

    else:
        X_std = X_parent[j].std(axis=0)
        temp_X = X_parent[j]/(X_std + 1e-8)
        xdawn = xdawn.fit(temp_X,Y[j])
        temp_X = xdawn.transform(temp_X)
        X[j] = np.hstack([temp_X,np.tile(xdawn.evokeds_[None,:,:],(temp_X.shape[0],1,1))])
        # X[j] = compute_riemannian_alignment(X[j], mean=None, dtype='real')


# DA
X_best, Y_best, domains_best = get_most_corr(X,Y,domains,i,max_sub)

(24, 8325, 8, 175)
intrresting corr0 [ 0.63587326  0.66581505  0.58401926  0.26961039 -0.24234214  0.16683386
 -0.00680923 -0.34201747]
intrresting corr1 [-4.16710703e-01 -1.87357815e-01 -3.18098970e-01 -3.11815943e-01
 -1.36594268e-01  4.09762651e-04  3.01444245e-01  2.72948948e-01]
intrresting corr2 [ 0.63897895 -0.38744962 -0.18318406 -0.23929243 -0.1145253  -0.25729512
 -0.10166816  0.15324004]
intrresting corr3 [-0.68356471 -0.68715853  0.11004254  0.08184281 -0.22914689 -0.3149597
  0.34355575 -0.11566502]
intrresting corr4 [-0.76533363  0.1910009   0.08576803 -0.39036946 -0.01079042  0.09439095
  0.2393305   0.03219436]
intrresting corr5 [-9.05662785e-01 -3.47962382e-03 -6.09650694e-02 -2.18560233e-01
  2.93439319e-01 -5.38186296e-01  7.64442454e-03 -6.22480967e-04]
intrresting corr6 [-0.91845275  0.20629646  0.07510972  0.3071451   0.15752127  0.03962606
 -0.08868562 -0.23267801]
intrresting corr7 [-0.84958128  0.45138827  0.02703538  0.20786834 -0.08241827  0.07691446
  0.3467

In [147]:
# cosine_similarity(X_parent[0][Y_parent[0]==1],X_parent[1][Y_parent[0]==1])
from sklearn.metrics.pairwise import cosine_similarity
X1 = X_parent[6][Y_parent[0]==1]
X2 = X_parent[3][Y_parent[0]==1]
X_std = X1.std(axis=0)
X1 = X1/(X_std + 1e-8)
X_std = X2.std(axis=0)
X2 = X2/(X_std + 1e-8)
cs = cosine_similarity(np.reshape(X1,(X1.shape[0],X1.shape[1]*X1.shape[2])),np.reshape(X2,(X2.shape[0],X2.shape[1]*X2.shape[2])))
np.where(np.mean(cs,axis=0)>0.01)[0].shape

(141,)

In [83]:
cosine_similarity([[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[1,4,6],[7,8,9]])

array([[1.        , 0.99120118, 0.95941195],
       [0.97463185, 0.93922134, 0.99819089],
       [0.95941195, 0.91715798, 1.        ]])

In [15]:
from skada import CORALAdapter, make_da_pipeline,EntropicOTMappingAdapter
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [17]:
keras.backend.clear_session()
CNN_accuracy_code_loo = np.zeros(12)
CNN_tps_train_code_loo = np.zeros(12)
CNN_tps_test_code_loo = np.zeros(12)
CNN_accuracy_loo = np.zeros(12)
freq = 60

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    
    n_cal = 4
    nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*60)
    ind2take = [j for j in range(len(participants)) if j!=i]
    X_train = np.concatenate([np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_sample_cal]]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate([np.concatenate(Y[ind2take]).reshape(-1),Y[i][:nb_sample_cal]]).reshape(-1)
    X_test = X[i][nb_sample_cal:]
    Y_test = Y[i][nb_sample_cal:]
    labels_code_test = labels_code_list[i][(n_class*4):]
    domains_train = np.concatenate([np.concatenate(domains[ind2take]).reshape(-1),-1*domains[i][:nb_sample_cal]]).reshape(-1)
    domain_test = -1*domains[i][nb_sample_cal:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = make_da_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),
            TangentSpace(),EntropicOTMappingAdapter(),
            LDA(solver="lsqr", shrinkage="auto"))
    x_train, x_val, y_train, y_val, domains_train,domains_val = train_test_split(X_train, Y_train,domains_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    
    clf.fit(x_train, y_train, sample_domain=domains_train) # sample_domain passed by name

    CNN_tps_train_code_loo[i] = time.time() - start

    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_loo[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    print("getting accuracy of participant ", i)
    print("the accuracy is ",CNN_accuracy_loo[i])

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
    )
    CNN_tps_test_code_loo[i] = time.time() - start
    CNN_accuracy_code_loo[i] = np.round(balanced_accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_loo)
print(CNN_accuracy_code_loo)
print(CNN_tps_train_code_loo)
print(CNN_tps_test_code_loo)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score/LOO_score",CNN_accuracy_loo)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score_code/LOO_score_code",CNN_accuracy_code_loo)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/temps_code/LOO_tps_train_code",CNN_tps_train_code_loo)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/temps_code/LOO_tps_test_code",CNN_tps_test_code_loo)


TL to the participant :  0
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
the accuracy is  0.5304142927016273
TL to the participant :  1
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
the accuracy is  0.5637830978095628
TL to the participant :  2
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
the accuracy is  0.6136760215210121
TL to the participant :  3
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
the accuracy is  0.5262336906949383
TL to the participant :  4
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of p

c:\Users\s.velut\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  6
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  6
the accuracy is  0.588727907253427
TL to the participant :  7
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  7
the accuracy is  0.6206723333377397
TL to the participant :  8
(193695, 8, 175)
(6105, 8, 175)
balancing the number of ones and zeros
Creating the different pipelines
Fitting


c:\Users\s.velut\AppData\Local\anaconda3\Lib\site-packages\ot\backend.py:1150: RuntimeWarning: overflow encountered in exp
  return np.exp(a)
c:\Users\s.velut\AppData\Local\anaconda3\Lib\site-packages\ot\da.py:1225: RuntimeWarning: invalid value encountered in divide
  transp_Xs_ = nx.dot(K, self.xt_) / nx.sum(K, axis=1)[:, None]


ValueError: Input X contains NaN.
LinearDiscriminantAnalysis does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [49]:
-1*domains_parent[0][nb_sample_cal:]

array([-1, -1, -1, ..., -1, -1, -1])